Load in sample clips, append a filepath and remove alle other columns.


In [1]:
import pandas as pd
import glob 

In [2]:
def load_all_csv_to_df(directory_path):
    # Create a pattern for glob to match all CSV files in the directory
    csv_pattern = f"{directory_path}/*.csv"
    
    # Use glob to get a list of CSV files in the directory
    csv_files = glob.glob(csv_pattern)
    
    # Initialize an empty DataFrame to merge all others into
    combined_df = pd.DataFrame()
    
    # Iterate over the list of CSV files
    for file in csv_files:
        # Read the current CSV file into a DataFrame
        current_df = pd.read_csv(file)
        
        # If combined_df is empty, just copy the current_df to it
        if combined_df.empty:
            combined_df = current_df
        else:
            # Merge the current DataFrame with the combined DataFrame on the "file" column
            # Use an outer join to ensure no data is lost
            combined_df = pd.merge(combined_df, current_df, on="file", how="outer")

    return combined_df

In [3]:
# load in sample_clips
sample_clips = pd.read_csv("../../data/sample_clips.csv", delimiter=";")

save_paths = []
for index, row in sample_clips.iterrows():
    start_min_formatted = row['start_min'].replace(':', '-')
    save_path = f"{row['name']}_video_{row['video_id']}_start_{start_min_formatted}"
    save_paths.append(save_path)

#add save paths to df
sample_clips['file'] = save_paths

#keep only relevant columns
sample_clips = sample_clips[['file', 'text']]
# rename text column to folketinget
sample_clips = sample_clips.rename(columns={"text": "folketinget"})

In [4]:
results = load_all_csv_to_df("../../data/models/")
results = results[['file', 'whisper-tiny', 'whisper-small', 'whisper-medium', 'whisper-large-v2', 'hviske']]

In [5]:
# merge with sample_clips on file
results = pd.merge(sample_clips, results, on="file")
# sort file alphabetically
results = results.sort_values(by="file")

In [6]:
results

,file,folketinget,whisper-tiny,whisper-small,whisper-medium,whisper-large-v2,hviske
83,Alexander Ryle_video_40_start_3-42-15,Mange tak. Når et byggeri eller etableringen a...,Og med byggeri eller et tablering af infrastr...,Om et byggeri eller etablering af infrastrukt...,Om et byggeri eller etablering af infrastrukt...,Om et byggeri eller etablering af infrastrukt...,Og om et byggeri eller etablering af infrastru...
82,Alexander Ryle_video_40_start_3-45-24,"Altså, det er jo vores generelle opfattelse, a...",Der er det jo vores generellt opfærdelsværdet...,"Det er vores generelle opfattelse, at et lov,...","Det er vores generelle opfattelse, at loven, ...","Det er vores generelle opfattelse, at loven, ...","Det er jo vores generelle opfattelse, at loven..."
78,Anders Kronborg_video_41_start_3-38-46,Først og fremmest vil jeg sige tak for en god ...,"Så var det en god vorte, at det var en afmæst...","Så frem, synes jeg, at tak for en god overfør...","Jeg synes, at jeg er tak for en god ordførtal...","Jeg vil bare fremme, at jeg synes, at jeg er ...","Fremad synes jeg tak for en god ordførertale, ..."
79,Anders Kronborg_video_41_start_4-33-32,"Tak for det, og tak for en, synes jeg, tale, d...",I en tag for det er og tagt synes jeg også fo...,"Tak for det, og tak synes jeg også for en tal...","Jamen tak for det, og tak synes jeg også for ...","Tak for det, og tak, synes jeg også, for en t...","Jamen, tak for det, og tak, synes jeg, også fo..."
47,Betina Kastbjerg_video_44_start_4-22-20,"Tak for ordet, formand. Beslutningsforslaget, ...","Bistotusforslædelsen viske drøft i dag, det e...","Beslutningsforslaget, som vi skal drifte i da...","Beslutningsforslaget, som vi skal drøfte i da...","Beslutningsforslaget, som vi skal drøfte i da...","Beslutningsforslaget, som vi skal drøfte i dag..."
...,...,...,...,...,...,...,...
60,Steffen Larsen_video_44_start_3-24-01,"Tak for det, hr. formand. Beslutningsforslaget...","Det er en slødningsvorslede, vi behandler her...",Beslutningsforslaget vi behandler her om hand...,Beslutningsforslaget omhandler gennemførelsen...,Beslutningsforslaget vi behandler her omhandl...,"Beslutningsforslaget, vi behandler her, omhand..."
86,Søren Egge Rasmussen_video_40_start_0-24-56,"Jeg vil altså hellere have, at det er Folketin...",Hvad har det folket tænkte at styr udviklinge...,Det er en grundholdning. I sætter man har et ...,"Det er ikke folketingen, der styrer udvikling...","Jeg vil gerne have, at det er Folketinget, de...","Jeg vil gerne have det i folketinget, der stør..."
85,Søren Egge Rasmussen_video_40_start_1-09-48,Tak. Der var nogle gode intentioner i minister...,"Jeg tager, der var nogle god en centrum, der ...","Ja tak. Når Ministeren siger, at det er et pl...","Ja tak. Når ministeren siger, at det er et pl...",Der var nogle gode intentioner i ministerens ...,Jeg tak.Der var nogle gode intentioner i minis...
87,Søren Egge Rasmussen_video_40_start_2-32-23,"Jo, med hensyn til herregårdslandskaber er der...","Læmpe som er der ikke, vil stærse den kommune...","landskaber. Der er sket en læmpelse, men der ...","Der er jo ikke sket en lempelse, hvis en komm...","Der er sket en lempelse, men der er jo ikke s...","I vore landskaber er der sket en lempelse, hvi..."


In [7]:
#save the df
results.to_csv("../../data/models/combined_results.csv", sep=",", index=False)